In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc", cache_dir='./cache')


In [14]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [16]:
from huggingface_hub import login
login()

There are some real issues with tqdm and huggingface hub!
I can't get the line below to work in a notebook, it just hangs trying to acquire a TQDM lock.
But if you run it outside the notebook, everything works fine.
So we're going to run it as a shell command for the initial download.

In [17]:
%%bash
set -euo pipefail

python - <<'PY'
from transformers import AutoModelForSequenceClassification, AutoConfig
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
PY

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now that it is downloaded we can run it in the notebook.

In [18]:
from transformers import AutoModelForSequenceClassification, AutoConfig

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
import torch

In [20]:
torch.cuda.is_available()

True

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

In [22]:
trainer.train()

Step,Training Loss
500,0.572800
1000,0.383400


TrainOutput(global_step=1377, training_loss=0.4222035459963065, metrics={'train_runtime': 92.5113, 'train_samples_per_second': 118.948, 'train_steps_per_second': 14.885, 'total_flos': 405114969714960.0, 'train_loss': 0.4222035459963065, 'epoch': 3.0})

In [23]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [24]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [26]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8480392156862745, 'f1': 0.8952702702702703}

In [27]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch", num_train_epochs=5)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.676510,0.696078,0.817109
2,0.621100,0.532288,0.818627,0.875839
3,0.513900,0.548588,0.803922,0.861592
4,0.387000,0.732075,0.803922,0.864407
5,0.298100,0.851883,0.791667,0.853701


TrainOutput(global_step=2295, training_loss=0.4239462443166828, metrics={'train_runtime': 163.3973, 'train_samples_per_second': 112.242, 'train_steps_per_second': 14.046, 'total_flos': 675891190117440.0, 'train_loss': 0.4239462443166828, 'epoch': 5.0})

In [32]:
training_args = TrainingArguments("test-trainer", 
                                  eval_strategy="epoch",
                                  num_train_epochs=3,
                                 fp16=True)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.404944,0.821078,0.868468
2,0.524300,0.432226,0.865196,0.904348
3,0.294800,0.594649,0.865196,0.906303


TrainOutput(global_step=1377, training_loss=0.3420354805740656, metrics={'train_runtime': 102.6814, 'train_samples_per_second': 107.166, 'train_steps_per_second': 13.41, 'total_flos': 405114969714960.0, 'train_loss': 0.3420354805740656, 'epoch': 3.0})

In [34]:
training_args = TrainingArguments("test-trainer", 
                                  eval_strategy="epoch",
                                  num_train_epochs=3,
                                  fp16=True,
                                 lr_scheduler_type="cosine")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.520186,0.757353,0.842105
2,0.590300,0.518858,0.816176,0.873524
3,0.415000,0.688941,0.830882,0.882051


TrainOutput(global_step=1377, training_loss=0.4446430580288756, metrics={'train_runtime': 90.9402, 'train_samples_per_second': 121.003, 'train_steps_per_second': 15.142, 'total_flos': 405114969714960.0, 'train_loss': 0.4446430580288756, 'epoch': 3.0})